# QA Task Evaluation

## Import Libaries & Basic Setting

In [54]:
# libraries
import os
import pandas as pd
import nest_asyncio
from dotenv import load_dotenv

from autorag.evaluator import Evaluator

from autorag.utils import fetch_contents
import itertools

from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.metrics import Faithfulness, NoiseSensitivity
from ragas import evaluate

from autorag.utils.util import to_list
from deepeval.dataset import EvaluationDataset
from deepeval.test_case import LLMTestCase
from deepeval import evaluate
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric

In [2]:
nest_asyncio.apply()

In [3]:
load_dotenv()

True

In [4]:
root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")
config_dir = os.path.join(root_dir, "config")
project_dir = os.path.join(root_dir, "autorag_project")

## G-Eval Evaluation with AutoRAG

In [5]:
data_path = os.path.join(data_dir, "qa_after_prompt_maker.parquet")

df = pd.read_parquet(data_path)

In [6]:
df.head()

,qid,query,retrieval_gt,generation_gt,queries,retrieved_contents,retrieved_ids,retrieve_scores,retrieval_f1,retrieval_ndcg,retrieval_map,prompts,prompt_maker_rouge,prompt_maker_sem_score
0,ec0c2280-9e54-4f6a-ad1a-56fea42d135e,뉴진스가 데뷔 후 몇 개월 만에 도쿄돔에서 공연을 했습니까?,[[71492e0b-50a8-4bfb-8c9d-6a2ff427fc5a]],"[뉴진스는 데뷔 23개월 만에 도쿄돔에서 공연을 했습니다., 23개월 만에]",[뉴진스가 데뷔 후 몇 개월 만에 도쿄돔에서 공연을 했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[068b8cb8-611d-4838-bfd7-403af9003b61, e2bfffd...","[14.79356829419634, 12.205198861234598, 10.807...",0.0,0.0,0.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,0.0,0.966844
1,32a73a5d-57b3-4fec-9b2c-924eef16ee02,뉴진스가 도쿄돔에서 공연을 한 시점은 데뷔 후 몇 개월입니까?,[[ae30d25f-943e-4a39-9d28-e7e51611a610]],"[뉴진스가 도쿄돔에서 공연을 한 시점은 데뷔 후 23개월입니다., 23개월]",[뉴진스가 도쿄돔에서 공연을 한 시점은 데뷔 후 몇 개월입니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[e2bfffd2-2384-4eae-b266-d98c5892a301, 068b8cb...","[14.442687515565346, 13.954902875800917, 12.50...",0.0,0.0,0.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,1.0,1.000000
2,76e58068-1054-4447-bc2e-65b2d979c4d5,뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?,[[e2bfffd2-2384-4eae-b266-d98c5892a301]],[뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 26일과 27일에 개최했습니다....,[뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[e2bfffd2-2384-4eae-b266-d98c5892a301, 063b69b...","[15.497493300020322, 15.310706007336684, 14.49...",0.5,1.0,1.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,1.0,0.997347
3,def11ba1-78e7-4741-80ef-f9d947fd576e,뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?,[[d35d8977-3ade-4c43-80a7-db166a1954db]],[The provided text does not contain informatio...,[뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[e2bfffd2-2384-4eae-b266-d98c5892a301, 063b69b...","[15.497493300020322, 15.310706007336684, 14.49...",0.0,0.0,0.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,0.0,0.814326
4,d93d7f40-cef2-4877-866e-8e54501e1c39,뉴진스의 도쿄돔 공연에서 몇 곡을 공연했습니까?,[[1c1ab88b-a641-42e0-b48d-c00c6cd425fc]],"[뉴진스는 도쿄돔 공연에서 총 22곡을 공연했습니다., 22곡]",[뉴진스의 도쿄돔 공연에서 몇 곡을 공연했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[1c1ab88b-a641-42e0-b48d-c00c6cd425fc, e2bfffd...","[14.663931356093451, 12.872650531476994, 12.59...",0.5,1.0,1.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,1.0,0.999999


In [7]:
df.shape

(20, 14)

In [8]:
g_eval_yaml_path = os.path.join(config_dir, "g_eval.yaml")
qa_path = os.path.join(data_dir, 'qa_after_prompt_maker.parquet')
corpus_path = os.path.join(data_dir, 'corpus.parquet')

In [9]:
evaluator = Evaluator(
    qa_data_path = qa_path,
    corpus_data_path = corpus_path,
    project_dir = project_dir
)

In [10]:
evaluator.start_trial(g_eval_yaml_path)

                    INFO     [evaluator.py:127] >>                                                 ]8;id=666045;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py\evaluator.py]8;;\:]8;id=993619;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py#127\127]8;;\
                                             _        _____            _____                                       
                                  /\        | |      |  __ \     /\   / ____|                                      
                                 /  \  _   _| |_ ___ | |__) |   /  \ | |  __                                       
                                / /\ \| | | | __/ _ \|  _  /   / /\ \| | |_ |                                      
                               / ____ \ |_| | || (_) | | \ \  / ____ \ |__| |                                      
                              /_/    \_\__,_|\__\___/|_|  \_\/_/    \_\_____|                                      
                                                                                                                   
                                                                                                                   

                    INFO     [evaluator.py:128] >> Start Validation input data and config YAML     ]8;id=613278;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py\evaluator.py]8;;\:]8;id=116657;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py#128\128]8;;\
                             file first. If you want to skip this, put the --skip_validation flag                  
                             or `skip_validation` at the start_trial function.                                     

Output()

                    INFO     [evaluator.py:205] >> Running node line post_retrieve_node_line...    ]8;id=317255;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py\evaluator.py]8;;\:]8;id=850851;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py#205\205]8;;\

                    INFO     [node.py:55] >> Running node generator...                                   ]8;id=287268;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/schema/node.py\node.py]8;;\:]8;id=784876;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/schema/node.py#55\55]8;;\

...

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=748868;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=469985;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=178111;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=975375;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=717168;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=259889;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=827629;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=355827;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [evaluator.py:218] >> Evaluation complete.                            ]8;id=898102;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py\evaluator.py]8;;\:]8;id=918622;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/autorag/evaluator.py#218\218]8;;\

In [55]:
result_path = os.path.join(project_dir, "0", "post_retrieve_node_line", "generator", "best_0.parquet")

result = pd.read_parquet(result_path)

In [56]:
result.head()

,qid,query,retrieval_gt,generation_gt,queries,retrieved_contents,retrieved_ids,retrieve_scores,retrieval_f1,retrieval_ndcg,retrieval_map,prompts,prompt_maker_rouge,prompt_maker_sem_score,generated_texts,generated_tokens,generated_log_probs,g_eval
0,ec0c2280-9e54-4f6a-ad1a-56fea42d135e,뉴진스가 데뷔 후 몇 개월 만에 도쿄돔에서 공연을 했습니까?,[[71492e0b-50a8-4bfb-8c9d-6a2ff427fc5a]],"[뉴진스는 데뷔 23개월 만에 도쿄돔에서 공연을 했습니다., 23개월 만에]",[뉴진스가 데뷔 후 몇 개월 만에 도쿄돔에서 공연을 했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[068b8cb8-611d-4838-bfd7-403af9003b61, e2bfffd...","[14.79356829419634, 12.205198861234598, 10.807...",0.0,0.0,0.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,0.0,0.966844,뉴진스는 데뷔 후 1년 11개월 만에 도쿄돔에서 공연을 했습니다. 이는 일본 이외 ...,"[38819, 13774, 5648, 2770, 40237, 8699, 8699, ...","[-0.0013816634891554713, -1.8624639324116288e-...",2.0
1,32a73a5d-57b3-4fec-9b2c-924eef16ee02,뉴진스가 도쿄돔에서 공연을 한 시점은 데뷔 후 몇 개월입니까?,[[ae30d25f-943e-4a39-9d28-e7e51611a610]],"[뉴진스가 도쿄돔에서 공연을 한 시점은 데뷔 후 23개월입니다., 23개월]",[뉴진스가 도쿄돔에서 공연을 한 시점은 데뷔 후 몇 개월입니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[e2bfffd2-2384-4eae-b266-d98c5892a301, 068b8cb...","[14.442687515565346, 13.954902875800917, 12.50...",0.0,0.0,0.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,1.0,1.000000,뉴진스가 도쿄돔에서 공연을 한 시점은 데뷔 23개월 만입니다. 단락에서는 뉴진스가 ...,"[38819, 13774, 5648, 4081, 27433, 8699, 8699, ...","[-0.0022645669523626566, -1.1472419600977446e-...",2.0
2,76e58068-1054-4447-bc2e-65b2d979c4d5,뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?,[[e2bfffd2-2384-4eae-b266-d98c5892a301]],[뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 26일과 27일에 개최했습니다....,[뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[e2bfffd2-2384-4eae-b266-d98c5892a301, 063b69b...","[15.497493300020322, 15.310706007336684, 14.49...",0.5,1.0,1.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,1.0,0.997347,뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 26일과 27일 이틀 동안 개최했...,"[38819, 13774, 5648, 2770, 97168, 27433, 8699,...","[-0.0006627569673582911, -9.088346359931165e-0...",3.0
3,def11ba1-78e7-4741-80ef-f9d947fd576e,뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?,[[d35d8977-3ade-4c43-80a7-db166a1954db]],[The provided text does not contain informatio...,[뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 언제 개최했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[e2bfffd2-2384-4eae-b266-d98c5892a301, 063b69b...","[15.497493300020322, 15.310706007336684, 14.49...",0.0,0.0,0.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,0.0,0.814326,뉴진스는 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 26일과 27일에 개최했습니다. ...,"[38819, 13774, 5648, 2770, 97168, 27433, 8699,...","[-0.0007148159202188253, -9.088346359931165e-0...",2.0
4,d93d7f40-cef2-4877-866e-8e54501e1c39,뉴진스의 도쿄돔 공연에서 몇 곡을 공연했습니까?,[[1c1ab88b-a641-42e0-b48d-c00c6cd425fc]],"[뉴진스는 도쿄돔 공연에서 총 22곡을 공연했습니다., 22곡]",[뉴진스의 도쿄돔 공연에서 몇 곡을 공연했습니까?],"[파일 제목: [현장] 데뷔 23개월 만에 도쿄돔 달군 뉴진스, 월드투어 기대감 키...","[1c1ab88b-a641-42e0-b48d-c00c6cd425fc, e2bfffd...","[14.663931356093451, 12.872650531476994, 12.59...",0.5,1.0,1.0,단락을 읽고 질문에 답하세요. 답할때 단계별로 천천히 고심하여 답변하세요. 반드시 ...,1.0,0.999999,뉴진스의 도쿄돔 공연에서 그들은 총 22곡을 공연했습니다. 공연의 마지막 곡은 '디...,"[38819, 13774, 5648, 3408, 27433, 8699, 8699, ...","[-0.001603771117515862, -1.0280383548888494e-0...",1.0


In [13]:
def show_result(df, idx: int):
    print(
        f"Query : {df.iloc[idx]['query']}",
        f"\n\nGround Truth : {df.iloc[idx]['generation_gt']}",
        f"\n\nPredicted : {df.iloc[idx]['generated_texts']}",
        f"\n\nG-Eval : {df.iloc[idx]['g_eval']}"
    )

In [14]:
show_result(result, 0)

Query : 뉴진스가 데뷔 후 몇 개월 만에 도쿄돔에서 공연을 했습니까? 

Ground Truth : ['뉴진스는 데뷔 23개월 만에 도쿄돔에서 공연을 했습니다.' '23개월 만에'] 

Predicted : 뉴진스는 데뷔 후 1년 11개월 만에 도쿄돔에서 공연을 했습니다. 이는 일본 이외 아티스트 중에서 데뷔 이후 최단 기간에 도쿄돔에 입성한 기록입니다. 따라서, 질문에 대한 답은 "1년 11개월"입니다. 

G-Eval : 2.0


In [15]:
show_result(result, 8)

Query : 뉴진스가 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 개최한 날짜는 언제입니까? 

Ground Truth : ["뉴진스가 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 개최한 날짜는 지난 26일과 27일입니다." '지난 26~27일.'] 

Predicted : 뉴진스가 일본 도쿄돔에서 팬미팅 '버니즈 캠프'를 개최한 날짜는 26일과 27일입니다. 

G-Eval : 2.0


In [16]:
show_result(result, 10)

Query : 뉴진스가 일본 도쿄돔에서 팬미팅 '버니즈 캠프' 공연을 펼친 날짜는 언제입니까? 

Ground Truth : ["뉴진스가 일본 도쿄돔에서 팬미팅 '버니즈 캠프' 공연을 펼친 날짜는 26일과 27일입니다." '26~27일'] 

Predicted : 뉴진스가 일본 도쿄돔에서 팬미팅 '버니즈 캠프' 공연을 펼친 날짜는 26일과 27일입니다. 이 사실은 여러 문장에 명시되어 있으며, 이틀 동안의 공연이 있었다고 설명되어 있습니다. 

G-Eval : 1.0


## Evaluate with RAGAS

In [33]:
# Transform AutoRAG formed data into RAGAS form
def transform_data_format_autorag_into_ragas(
    autorag_df : pd.DataFrame,
    corpus_df : pd.DataFrame
)-> EvaluationDataset:
    autorag_df["retrieval_gt_contents"] = autorag_df["retrieval_gt"].apply(
        lambda x: fetch_contents(corpus_df, x)
    )

    samples = []
    for idx, row in autorag_df.iterrows():
        samples.append(SingleTurnSample(
            user_input=row['query'],
            retrieved_contexts=row['retrieved_contents'],
            reference_contexts=list(
                itertools.chain.from_iterable( # for flatten 2-dim contents
                    row['retrieval_gt_contents']
                )
            ),
            response=row['generated_texts'],
            reference=row['generation_gt'][0]
        ))
    
    return EvaluationDataset(samples)

In [35]:
corpus_path = os.path.join(data_dir, "corpus.parquet")

corpus = pd.read_parquet(corpus_path)

In [36]:
ragas_dataset = transform_data_format_autorag_into_ragas(result, corpus)

In [37]:
print(f"Features in dataset : {ragas_dataset.features()}")
print(f"Total samples in dataset : {len(ragas_dataset)}")

Features in dataset : ['user_input', 'retrieved_contexts', 'reference_contexts', 'response', 'reference']
Total samples in dataset : 20


In [46]:
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [47]:
result = evaluate(
    ragas_dataset,
    metrics=[Faithfulness(
        name="Faithfulness",
        llm=ragas_llm
    ), NoiseSensitivity(
        name="NoiseSensitivity",
        llm=ragas_llm
    )]
)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

[05/23/25 15:21:28] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=241305;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=625667;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=747306;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=981027;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=706660;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=273929;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=823867;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=447854;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

[05/23/25 15:22:04] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=664236;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=449175;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/23/25 15:22:06] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=583162;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=939480;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=454607;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=203703;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/23/25 15:22:12] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=808374;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=18973;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/23/25 15:22:15] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=476571;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=7118;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [48]:
result

{'Faithfulness': 0.7895, 'NoiseSensitivity(mode=relevant)': 0.3458}

## Evaluate with DeepEval

In [71]:
def transform_data_format_autorag_into_deepeval(
    autorag_df: pd.DataFrame,
    corpus_df: pd.DataFrame
)-> EvaluationDataset:
    autorag_df["retrieval_gt_contents"] = autorag_df["retrieval_gt"].apply(
        lambda x: fetch_contents(corpus_df, x)
    )

    samples=[]
    for idx, row in autorag_df.iterrows():
        samples.append(LLMTestCase(
            input=row['query'],
            actual_output=row['generated_texts'],
            expected_output=row['generation_gt'][0],
            context=to_list(list(itertools.chain.from_iterable(row['retrieval_gt_contents']))),
            retrieval_context=to_list(row['retrieved_contents'])
        ))
    
    return EvaluationDataset(test_cases=samples)

In [72]:
corpus_path = os.path.join(data_dir, "corpus.parquet")
corpus = pd.read_parquet(corpus_path)

In [73]:
deepeval_dataset = transform_data_format_autorag_into_deepeval(result, corpus)

In [74]:
result = evaluate(
    deepeval_dataset,
    metrics=[
        FaithfulnessMetric(),
        AnswerRelevancyMetric()
    ]
)

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Evaluating 20 test case(s) in parallel: |          |  0% (0/20) [Time Taken: 00:00, ?test case/s]

[05/23/25 15:33:58] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=387022;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=547852;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=142894;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=9708;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

[05/23/25 15:34:06] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=593755;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=98461;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=813569;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=616543;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

Evaluating 20 test case(s) in parallel: |██████████|100% (20/20) [Time Taken: 00:09,  2.06test case/s]



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context. Great job maintaining accuracy and consistency!, error: None)
  - ✅ Answer Relevancy (score: 0.5, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.50 because while the output correctly identifies the title of the show, it includes irrelevant information about the broadcast time, which does not address the input question., error: None)

For test case:

  - input: 뉴진스가 출연할 예정인 두 번째 지상파 예능 프로그램의 제목은 무엇입니까?
  - actual output: 뉴진스가 출연할 예정인 두 번째 지상파 예능 프로그램의 제목은 KBS2의 ‘사장님 귀는 당나귀 귀’입니다. 이 프로그램은 매주 토요일 오후 4시 40분에 방송됩니다.
  - expected output: 사당귀
  - context: ["파일 제목: 뉴진스, 처음이 어렵지..두 번째 지상파 예능 출격 눈앞 ('사당귀')[Oh!쎈 예고] - 조선비즈.pdf\n 내용: /elnino8919@osen.<br>co.kr</p><h1 id='21' style='font-size:18px'>[OSEN=장우영 기자]</h1

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

In [79]:
result

EvaluationResult(test_results=[TestResult(name='test_case_6', success=True, metrics_data=[MetricData(name='Faithfulness', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context. Great job maintaining accuracy and consistency!', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.010065000000000001, verbose_logs='Truths (limit=None):\n[\n    "뉴진스는 \'1박 2일\' 이후 두 번째로 지상파 예능에 출연한다.",\n    "박명수는 지코와의 대화에서 라디오 고정 출연을 제안했다.",\n    "뉴진스는 박명수의 에피소드 중에 등장할 예정이다.",\n    "KBS2 \'사장님 귀는 당나귀 귀\'는 매주 토요일 오후 4시 40분에 방송된다.",\n    "박명수는 뉴진스와 챌린지를 했다.",\n    "지코는 박명수의 라디오 섭외 제안에 선뜻 답하지 않았다.",\n    "뉴진스는 대기실에서 멤버들과 포옹하며 친근함을 표시했다."\n] \n \nClaims:\n[\n    "뉴진스가 출연할 예정인 두 번째 지상파 예능 프로그램의 제목은 KBS2의 ‘사장님 귀는 당나귀 귀’입니다.",\n    "이 프로그램은 매주 토요일 오후 4시 40분에 방송됩니다."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "idk",\n        "reason": "The retrieval context 

In [78]:
result.test_results

[TestResult(name='test_case_6', success=True, metrics_data=[MetricData(name='Faithfulness', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context. Great job maintaining accuracy and consistency!', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.010065000000000001, verbose_logs='Truths (limit=None):\n[\n    "뉴진스는 \'1박 2일\' 이후 두 번째로 지상파 예능에 출연한다.",\n    "박명수는 지코와의 대화에서 라디오 고정 출연을 제안했다.",\n    "뉴진스는 박명수의 에피소드 중에 등장할 예정이다.",\n    "KBS2 \'사장님 귀는 당나귀 귀\'는 매주 토요일 오후 4시 40분에 방송된다.",\n    "박명수는 뉴진스와 챌린지를 했다.",\n    "지코는 박명수의 라디오 섭외 제안에 선뜻 답하지 않았다.",\n    "뉴진스는 대기실에서 멤버들과 포옹하며 친근함을 표시했다."\n] \n \nClaims:\n[\n    "뉴진스가 출연할 예정인 두 번째 지상파 예능 프로그램의 제목은 KBS2의 ‘사장님 귀는 당나귀 귀’입니다.",\n    "이 프로그램은 매주 토요일 오후 4시 40분에 방송됩니다."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "idk",\n        "reason": "The retrieval context does not specify the title of 

In [80]:
result.test_results[0]

TestResult(name='test_case_6', success=True, metrics_data=[MetricData(name='Faithfulness', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context. Great job maintaining accuracy and consistency!', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.010065000000000001, verbose_logs='Truths (limit=None):\n[\n    "뉴진스는 \'1박 2일\' 이후 두 번째로 지상파 예능에 출연한다.",\n    "박명수는 지코와의 대화에서 라디오 고정 출연을 제안했다.",\n    "뉴진스는 박명수의 에피소드 중에 등장할 예정이다.",\n    "KBS2 \'사장님 귀는 당나귀 귀\'는 매주 토요일 오후 4시 40분에 방송된다.",\n    "박명수는 뉴진스와 챌린지를 했다.",\n    "지코는 박명수의 라디오 섭외 제안에 선뜻 답하지 않았다.",\n    "뉴진스는 대기실에서 멤버들과 포옹하며 친근함을 표시했다."\n] \n \nClaims:\n[\n    "뉴진스가 출연할 예정인 두 번째 지상파 예능 프로그램의 제목은 KBS2의 ‘사장님 귀는 당나귀 귀’입니다.",\n    "이 프로그램은 매주 토요일 오후 4시 40분에 방송됩니다."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "idk",\n        "reason": "The retrieval context does not specify the title of t

In [81]:
result.test_results[0].metrics_data

[MetricData(name='Faithfulness', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context. Great job maintaining accuracy and consistency!', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.010065000000000001, verbose_logs='Truths (limit=None):\n[\n    "뉴진스는 \'1박 2일\' 이후 두 번째로 지상파 예능에 출연한다.",\n    "박명수는 지코와의 대화에서 라디오 고정 출연을 제안했다.",\n    "뉴진스는 박명수의 에피소드 중에 등장할 예정이다.",\n    "KBS2 \'사장님 귀는 당나귀 귀\'는 매주 토요일 오후 4시 40분에 방송된다.",\n    "박명수는 뉴진스와 챌린지를 했다.",\n    "지코는 박명수의 라디오 섭외 제안에 선뜻 답하지 않았다.",\n    "뉴진스는 대기실에서 멤버들과 포옹하며 친근함을 표시했다."\n] \n \nClaims:\n[\n    "뉴진스가 출연할 예정인 두 번째 지상파 예능 프로그램의 제목은 KBS2의 ‘사장님 귀는 당나귀 귀’입니다.",\n    "이 프로그램은 매주 토요일 오후 4시 40분에 방송됩니다."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "idk",\n        "reason": "The retrieval context does not specify the title of the second terrestrial variety show NewJeans will appear on

In [84]:
print(
    f"{result.test_results[0].metrics_data[0].name} : {result.test_results[0].metrics_data[0].score}",
    f"\n{result.test_results[0].metrics_data[1].name} : {result.test_results[0].metrics_data[1].score}"
)

Faithfulness : 1.0 
Answer Relevancy : 0.5
